For this tutorial, we will consider Ebay auctions of a video game called Mario Kart for the Nintendo Wii, which contains the total price of an auction (highest bid plus + the shipping cost) and certain characteristics of the  auction that may affect the final price.

In [25]:
library(tidyverse)
dat.mariokart<-read.csv("https://raw.githubusercontent.com/jrasero/cm-85309-2023/main/datasets/mariokart_ebay.csv")
dat.mariokart$cond<-as.factor(dat.mariokart$cond)
dat.mariokart$cond<-relevel(dat.mariokart$cond, ref = "used")

head(dat.mariokart)

,duration,n_bids,cond,total_pr,seller_rate,stock_photo,wheels
,<int>,<int>,<fct>,<dbl>,<int>,<chr>,<int>
1,3,20,new,51.55,1580,yes,1
2,7,13,used,37.04,365,yes,1
3,3,16,new,45.50,998,no,1
4,3,18,new,44.00,7,yes,1
5,1,20,new,71.00,820,yes,2
6,3,19,new,45.00,270144,yes,0


The 

<div class="alert alert-info">  <b>Practice</b>: Using <code>lm</code>, fit a regression model that aims to explain the predict total price based on using a stock photo game or not. Save this to an object named "reg.model.1".

In [26]:
# Your answer here

<div class="alert alert-info">  <b>Question</b>: Is the slope in "reg.model.1" significant at $\alpha=0.05$. How can this be interpreted?

In [51]:
# Your answer here

<div class="alert alert-info"> <b>Practice</b>: Now, fit two regression models, one that aims to explain the total price based on using a stock photo game and the number of wheels, and another one that aims to explain the total price based on using a stock photo game and game condition. Save these new objects to variables named "reg.model.2" and "reg.model.3" respectively.

In [ ]:
# Your answer here

<div class="alert alert-info">  <b>Question</b>: Is the slope for "stock_photo" in both models still significant at $\alpha=0.05$? If not, what do you think it happened?

In [1]:
# Your answer here

As we already knew, we use a multiple linear regression to be able to evaluate the relationship between an independent variable and the dependent variable while controlling for the potential influence of other variables. In other words:

- $H_0$: $β_i = 0$ when the other independent variables are included in the model.
- $H_a$: $β_i \neq 0$ when the other independent variables are included in the model.

Ok, so the answer is easy. We should fit a multiple linear regression model that includes all independent variables at hand. Is that right so? 

<div class="alert alert-info"> <b>Practice</b>: Fit a multiple linear regression model that aims to explain the total price based on all the remaining variables in the dataset. Save this model to an object called "reg.model.full". Then, fit another regression model that aims to explain the total price based on only "cond", "wheels", and "seller_rate". Save this model to an object called "reg.model.subset". If you had to choose one model, which would it be?

In [2]:
# Your answer here

The most complex model needs not to be the best one. The inclusion of unimportant variables can some times reduce the performance of the regression model. In fact, of any model that can make predictions. This is in general true for any predictive model, like linear regression...

As a general rule, there is a trade off between model's simplicity and goodness of fit. As new independent variables are added to the model, this becomes more complex: each independent variable adds a new free parameter (i.e., a new regression coefficient), and each new parameter increases the model's capacity to capture random variations that are not related to the problem in study. For a model to perform efficiently, one needs to select a model that do not contain too many variables. This is the so-called **model selection** problem.

To come up with a final model that fits the data well without being too complex, we need a measure that evaluates this balance. While we could use the adjusted $R^2$ for this purpose, we will instead introduce the **Akaike Information Criterion** (AIC) measure. The AIC for a model that has k predictor variables plus an intercept is as follows:

$$AIC = \frac{SS_{res}}{\hat{\sigma}^2} + 2k$$

The **smaller** the AIC value, the **better** the model performs. On the left we have $SS_{res}$, so if this increases, the model accounts for less data and therefore AIC increases; on the right, we have a term that increases as the model complexity increases. The best model is the one that fits the data well (low residuals; left hand side) using as few independent variables as possible (low k; right hand side).

Let's see a few strategies on how to select the most optimal model based on AIC. We are going to carry out this using the `step` R built-in function, which enables choosing a model by AIC in a stepwise fashion. 

In [86]:
?step

What do we mean by "stepwise"? We mean that at each step we will evaluate the effect of one variable in the entire model. This can be done either by looking at how that variable affects the model after deletion, or after inclusion. The former strategy is called **backward selection**, whereas the latter is called **forward selection**.

## Backward selection

This strategy for model selection can be summarized as follows:

1. Start with the full model.
2. Drop one variable at a time and record the AIC of each smaller model.
3. Pick the model with the highest decrease in AIC.
4. Repeat until none of the models yield a decrease in AIC.

To select the model using this kind of selection strategy in the `step` function, we just need to pass the full regression model and set the argument *direction* to "backward":

In [87]:
step( object = reg.model.full, direction = "backward")                 

Start:  AIC=441.4
total_pr ~ cond + wheels + stock_photo + duration + seller_rate + 
    n_bids

              Df Sum of Sq    RSS    AIC
- duration     1       0.8 2922.6 439.44
- n_bids       1      18.9 2940.8 440.31
- stock_photo  1      20.0 2941.8 440.36
<none>                     2921.8 441.40
- seller_rate  1     289.7 3211.5 452.73
- cond         1     449.6 3371.4 459.58
- wheels       1    4238.9 7160.8 565.79

Step:  AIC=439.44
total_pr ~ cond + wheels + stock_photo + seller_rate + n_bids

              Df Sum of Sq    RSS    AIC
- stock_photo  1      19.2 2941.8 438.36
- n_bids       1      20.1 2942.8 438.41
<none>                     2922.6 439.44
- seller_rate  1     290.8 3213.5 450.81
- cond         1     474.3 3396.9 458.64
- wheels       1    4403.2 7325.8 567.01

Step:  AIC=438.36
total_pr ~ cond + wheels + seller_rate + n_bids

              Df Sum of Sq    RSS    AIC
- n_bids       1      16.1 2957.9 437.13
<none>                     2941.8 438.36
- seller_rate  


Call:
lm(formula = total_pr ~ cond + wheels + seller_rate, data = dat.mariokart)

Coefficients:
(Intercept)      condnew       wheels  seller_rate  
  3.607e+01    4.728e+00    7.733e+00    3.103e-05  


In the above messages, the text in the left hand column is telling us what change R made to the regression model. The line that reads "< none >" is the actual model we started with. The other rows correspond to the other models after removing one of the independent variables. Each of these yields a new AIC. The one with the smallest AIC is the new selected model, and the backward selection process is run again but now on this smaller model.

This process is repeated until no further decrease in AIC is observed. In our case, this corresponds to the case in which we only have "cond", "wheels" and "seller_rate" as independent varibles, with AIC=437.13.

## Forward selection

This strategy for model selection can be summarized as follows:

1. Start by creating separate regression models for the dependent variable versus each independent variable.
2. Pick the model with the smallest AIC: Fitting more accurately the data will be more benefitial than the inclusion of new variable.
3. Add the remaining variables one at a time to the existing model, and once again pick the model smallest AIC.
4. Continue this process until no remaining variables decreases AIC.

We can carry out this selection strategy in the `step` function in the same way as with "backward". Now instead, we have to start with the simplest model possible, that is, one without any indepedent variable, set the argument *direction* to "forward", and specify what is the largest possible model we are willing to have, which can be done by means of the *scope* argument:

In [88]:
reg.model.null <- lm(total_pr~1, data = dat.mariokart)
scope<-total_pr~cond + wheels + stock_photo + duration + seller_rate + n_bids # Most complex model possible
step( object = reg.model.null,
     scope=scope,
     direction = "forward")

Start:  AIC=624.15
total_pr ~ 1

              Df Sum of Sq     RSS    AIC
+ wheels       1    7460.1  4168.1 481.49
+ cond         1    4076.3  7551.9 565.29
+ duration     1    1627.9 10000.3 604.89
+ stock_photo  1     466.0 11162.2 620.39
<none>                     11628.2 624.15
+ seller_rate  1     140.4 11487.8 624.44
+ n_bids       1      72.1 11556.1 625.28

Step:  AIC=481.49
total_pr ~ wheels

              Df Sum of Sq    RSS    AIC
+ cond         1    871.68 3296.4 450.41
+ seller_rate  1    620.99 3547.1 460.74
+ stock_photo  1    256.29 3911.8 474.54
+ duration     1    188.89 3979.2 476.95
<none>                     4168.1 481.49
+ n_bids       1      5.80 4162.3 483.29

Step:  AIC=450.41
total_pr ~ wheels + cond

              Df Sum of Sq    RSS    AIC
+ seller_rate  1    338.46 2957.9 437.13
<none>                     3296.4 450.41
+ n_bids       1     46.15 3250.2 450.42
+ stock_photo  1     28.69 3267.7 451.17
+ duration     1      4.06 3292.3 452.23

Step:  AIC=437


Call:
lm(formula = total_pr ~ wheels + cond + seller_rate, data = dat.mariokart)

Coefficients:
(Intercept)       wheels      condnew  seller_rate  
  3.607e+01    7.733e+00    4.728e+00    3.103e-05  


This process works in the same as above. It will stop when incorporating new variables does not decrease AIC.

**IMPORTANT:** As we can see, for this example the same model was found using both strategies. However, forward and backward selection do not need to always end up in the same place...

## Important caveat



# Mode comparison

An alternative to using automated model selection procedures is to explicitly select two or more regression models to compare to each other.


There are two different ways we can compare these two models, one based on a model selection criterion like AIC (or BIC if one wants to attach to Bayesian estimation - beyond the scope of this course), and the other based on an explicit hypothesis test.


## AIC-based

Let's start with the one based on AIC, which is very simple. Let's define two models, M0 and M1, that we want to compare one against each other:

In [90]:
M0 <- lm( total_pr ~ wheels + cond + seller_rate, data = dat.mariokart )
M1 <- lm( total_pr ~ ., data = dat.mariokart  )

We can pass these two (and any number of fitted objects) models to the `AIC` function in R, which will show the AIC values for each:

In [106]:
AIC(M0, M1)

,df,AIC
,<dbl>,<dbl>
M0,3,967.4329
M1,4,852.5476


As we can see, the model M0 has a smaller AIC, so this is the one we should select over M1 (Do not worry for the values of AIC here not matching the ones observed before. Here constant terms have been dropped, which are not important for model comparison).

## Hypohtesis testing-based

Suppose that we have got two regression models, one (M0) containing a subset of the independent variables from another model (M1), i.e.M1 contains all of the independent variables included in M0, plus one or more additional variables. In this case, we can think of M0 as a null hypothesis and M1 as an alternative hypothesis. 

For this situation, we can construct an F test that compares one model with respect to the other in the following way:

$$ F = \frac{(SS_{res}^0 - SS_{res}^1)/(k_1 - k_0)}{SS_{res}^1/(N-k_1-1)}$$

where $SS_{res}^0$ and $SS_{res}^1$ are the residual sum of squares for both models, $N$ is the number of observations, and $k_0$ and  $k_1$ are the number of independent variables in both models.

As a result, we can see that if SS_{res}^1 < SS_{res}^0, that is, the alternative model is able to fit better the data, the numerator increases and so does F. At the same time, since the numerator includes a factor for the difference in independent variables between both models, incresing the number of independent variables may not always be the best choice.

To compare two models in R based on hypothesis testing, we can use the `anova` function:

In [109]:
?anova

In [107]:
anova(M0, M1)

,Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,139,7551.908,NA,NA,NA,NA
2,138,3296.386,1,4255.523,178.1534,1.294506e-26


Since the p-value is higher than 0.05 (assuming this significance level for our statistical claims), we should stick with the null hypothesis model (M0). This approach to regression is usually called **hierarchical regression** and involves adding all the covariates into a null model, which serves as an alternative model, and then comparing the two models in a hypothesis testing framework.

<div class="alert alert-info"> <b>Practice</b>: Observe a case in which we should reject the null by comparing the regression model with only "cond" as the independent variable (M0) with the alternative model that also adds "wheel" (M1).

In [3]:
# Your answer here